# Climpy data
- Reading a NetCDF file, analyzing the internals and finally extract the dessired layer into new files (under development)
- WIth GDAL, extracting the bands one by one into new files (finished)

In [1]:
from osgeo import gdal
import netCDF4 as nc
from netCDF4 import num2date
import os

# concept of bands: https://stackoverflow.com/questions/52046282/convert-netcdf-nc-to-geotiff

In [2]:
def open_netcdf(data_path):
    ds = nc.Dataset(data_path, mode='r+') #, mode='w',format='NETCDF4'
    return ds

# Tip: longitude and latitute define the size of the layers. Time defines the number of layers inside 
def get_dimension_list(dataset):
    dimensions_list = list(dataset.dimensions.values())
    return dimensions_list

def get_variable_list(dataset):
    variables_list = list(dataset.variables.keys())
    return variables_list


def get_time_dimension_properties(ds):
    time_variable = ds.variables['time']
    print("time units:", time_variable.units)
    print("time calendar:", getattr(time_variable, 'calendar', 'standard'))
    print("time dimensions:", time_variable.dimensions)
    # Esto no sive pero me ha costado tanto que no quiero borrarlo
    days_since_yyyy_mm_dd = ' '.join(time_variable.units.split()[:3])  # Extract "days since YYYY-MM-DD" part
    dates = num2date(time_variable[:], units=time_variable.units, calendar=getattr(time_variable, 'calendar', 'standard'))

    time_min, time_max = min(dates), max(dates)
    print(f'Time range: {time_min} to {time_max}')

def get_date_list(ds):
    date_list = []
    dimension = 'time'
    for dim in ds.dimensions.values():
        if dim == ds.dimensions[dimension]:
            print("number of date bands: ", len(dim))
            variable = ds.variables[dimension]
            starting_date = variable.units
            array = variable[:]
            time_format = nc.num2date(array,starting_date)
            for date_element in time_format:
                # replace "-" for " " so "split" can work, filter and create a list by digits and finally join them in a single string with "_" as separator
                date_element_formated = "_".join([str(s) for s in str(date_element).replace("-"," ").split() if s.isdigit()])
                date_list.append(date_element_formated)
        else:
            continue
    return date_list

def create_output_folder(path):
    root_dir = os.path.dirname(path)
    output_folder = "netcdf_output"
    output_dir = os.path.join(root_dir, output_folder).replace("\\","/")
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    return output_dir

def netcdf_to_geotiff(data_path, output_dir, variable, date_list):
    gdal.UseExceptions()

    file_name = os.path.basename(data_path)
    print(file_name)
    k = 0
    for i in date_list: # [0:2] "for testing"
        k += 1
        output = output_dir + "/" + file_name.replace(".nc","") + "_" + i + ".tif"
        print(output)
        """Method1"""
        # gdal.Translate(output, fn, 
        #                 format='NetCDF', # format='GTIFF',
        #                 bandList = [1],
        #                 creationOptions = ['COMPRESS=DEFLATE','TILED=YES']
        #                 )
        """Method2"""
        gdaltranString = "gdal_translate -of GTiff -a_srs epsg:4326 -b " + str(k) + " -co COMPRESS=DEFLATE -co TILED=YES NETCDF:" + str(data_path) + f":{variable} " + str(output)
        os.system(gdaltranString)
        output = None

    # gdal_translate -of GTiff -b 10 input.nc output.tiff  # to get 10th band, it works fine



In [3]:
data_path = r"C:\Users\admin\Downloads\reg_rcp85_17g_accretion.nc"

try: 
    ds.close()  # just to be safe, make sure dataset is not already open.
except: 
    pass

ds = open_netcdf(data_path)
dimensions_list = get_dimension_list(ds)
variables_list = get_variable_list(ds)
# get_time_dimension_properties(ds)

In [4]:
variables_list

['Accretion', 'transverse_mercator', 'StdTime', 'x', 'y']

In [9]:
output_dir = create_output_folder(data_path)
date_list = get_date_list(ds)
variable = "lccs_class"
netcdf_to_geotiff(data_path, output_dir, variable, date_list)

number of date bands:  1
C3S-LC-L4-LCCS-Map-300m-P1Y-2022-v2.1.1.nc
C:/Users/admin/Downloads/dataset-satellite-land-cover/netcdf_output/C3S-LC-L4-LCCS-Map-300m-P1Y-2022-v2.1.1_2022_01_01.tif


c:\Users\admin\.conda\envs\gdal_env\lib\site-packages\ipykernel_launcher.py:35: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
